In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
import matplotlib.pyplot as plt
import openpyxl
import pandas as pd
from matplotlib import image
import cv2
import numpy as np
from mlxtend.evaluate import mcnemar_table, mcnemar
from mlxtend.plotting import checkerboard_plot


from numpy.random import seed
seed(2)

import tensorflow
tensorflow.random.set_seed(2)    

class DelongTest:
    
    
    def __init__ (self, 
                  problem, 
                  path_csv, path_images, img_width, img_height, type_image, 
                  model_hybrid, model_images, model_directory):
    
        self.type_problem = problem
        self.path_csv = path_csv
        self.path_images = path_images
        self.img_width= img_width
        self.img_height= img_height
        
        #test_delong
        self.path_model_hybrid=model_hybrid
        self.path_model_images=model_images
        self.type_image= type_image
        self.model_directory = model_directory
        
    
    def load_data_ICH(self):
        if not os.path.exists(self.model_directory + 'Input/'): os.mkdir(self.model_directory + 'Input/')  

        self.partition='Test'
        self.X_test_image, self.y_test  = self.load_imagesICH()
        self.X_test_dataf = self.load_dataframe_ICH()
           
        print('[INFO] DEMOGRAFIC DATA TEST SHAPE: ' + str(self.X_test_dataf.shape))
        print('[INFO] IMAGES TEST SHAPE: ' + str(self.X_test_image.shape))
        print('[INFO] LABELS TEST SHAPE: ' + str(self.y_test.shape))
        
        
    def load_imagesICH(self):  
        if self.type_image=='3D':
            self.csv = pd.read_csv(self.path_csv + 'TEST15_PATIENT_NUEVO_random_normalizado.csv', sep=',') 
                
        elif self.type_image=='2D':
            self.csv = pd.read_csv(self.path_csv + 'TEST15_NUEVOS.csv', sep=',') 
            
        X = []
        for filename in self.csv['Patient']:
            if self.type_image == '3D':
                img_data = np.load(self.path_images + filename +'.npy')
                img_data = self.resize(img_data)
                    
            elif self.type_image == '2D':
                # load image
                img_data = image.imread(self.path_images + filename)
                img_data = self.resize(img_data)
                    
            X.append(img_data)

        X = np.expand_dims(np.asarray(X, dtype=np.float64), axis=4) if self.type_image == '3D' else np.stack((X,X,X), axis=3).astype('float64')
        return X, self.csv['Label']
    
    
    def load_dataframe_ICH(self): 
        for i in range(len(self.csv['Patient'])): 
            self.csv.loc[i, 'PatientAge'] = round((self.csv['PatientAge'][i]-min(self.csv['PatientAge'])) / (max(self.csv['PatientAge']) - min(self.csv['PatientAge'])),2)
        self.csv_data = self.csv
        self.csv_data=self.csv_data.drop(['Patient','Label'], axis=1)
        self.csv_data.to_csv(self.model_directory + 'Input/'+self.partition+'_data.csv', index = False)
        return np.asarray(self.csv_data).astype('float64')
    
    
    def load_data_PROGNOSIS(self): 
        if not os.path.exists(self.model_directory + 'Input/'): os.mkdir(self.model_directory + 'Input/')   
        
        self.partition='Test'
        self.X_test_image, self.y_test  = self.load_images_PROGNOSIS()
        self.X_test_dataf = self.load_dataframe_PROGNOSIS()
        
        print('[INFO] CLINICAL DATA TEST SHAPE: ' + str(self.X_test_dataf.shape)) 
        print('[INFO] IMAGES TEST SHAPE: ' + str(self.X_test_image.shape))
        print('[INFO] LABELS TEST SHAPE: ' + str(self.y_test.shape))
            
        
        
    def load_images_PROGNOSIS(self): 
        self.csv = pd.read_csv(self.path_csv + 'TEST.csv', sep=',') 
            
        X = []
        for filename in self.csv['Marca temporal']:
            img_data = np.load(self.path_images + 'HIC_'+str(filename) +'.npy')
            img_data = self.resize(img_data)

            X.append(img_data)
        X = np.expand_dims(np.asarray(X, dtype=np.float64), axis=4)
            
        return X, np.asarray(self.csv['Muerte']).astype('float64')
    
    
    def load_dataframe_PROGNOSIS(self): 
        csv_data = self.csv
        for column in csv_data.columns.tolist():                
            for i in range(len(self.csv[column])): 
                csv_data.loc[i, column] = round((csv_data[column][i]-min(csv_data[column])) / (max(csv_data[column]) - min(csv_data[column])),2)
        list_to_ignore=[]
        ignore=0
        columns=csv_data.columns
        while ignore != "None":
            ignore = str(input("\n What column do you want to ignore? - None = Break - (options: "+ str(columns.tolist())+"): "))
            if ignore != "None": 
                list_to_ignore.append(ignore)
                columns = columns.drop(ignore)
            
        self.list_to_ignore=list_to_ignore
        csv_data=csv_data.drop(self.list_to_ignore, axis=1)   
        
        print("\n You ignored for :"+str(self.list_to_ignore))
        
        csv_data.to_csv(self.model_directory + 'Input/'+self.partition+'_data.csv', index = False)
        return np.asarray(csv_data).astype('float64')
 
        
    def resize(self, img_data):    
        if self.type_image=='3D':
            n,m,l=img_data.shape
            img_p=list(img_data)

            matriz_float = np.zeros((m,l))
            matriz_float = np.float32(matriz_float)
            self.depth=45
            for ind in range(n,self.depth):
                img_p.append(matriz_float)
            img_p=np.array(img_p)


            n,m,l=img_p.shape
            img_data=np.zeros([self.img_width,self.img_height, n])

            for i in range(0,n):
                img_data[:,:,i]=cv2.resize(img_p[i,:,:],(self.img_width,self.img_height,))
                
        elif self.type_image == '2D':
            img_data = cv2.resize(img_data, (self.img_width, self.img_height))
            img_data = img_data/255.
            if len(img_data.shape) == 3: img_data=img_data[:,:,0]

        return img_data
    
        
    def load_models_delong(self):
        if not os.path.exists(self.model_directory +'/TEST_DELONG/'): os.mkdir(self.model_directory +'/TEST_DELONG/') 
        if not os.path.exists(self.model_directory +'/TEST_DELONG/'+self.type_problem +'/'): os.mkdir(self.model_directory +'/TEST_DELONG/'+self.type_problem +'/') 
        if not os.path.exists(self.model_directory +'/TEST_DELONG/'+self.type_problem +'/' +self.type_image+'/'): os.mkdir(self.model_directory +'/TEST_DELONG/'+self.type_problem +'/' +self.type_image+'/') 
        
        name_model = self.path_model_hybrid.split('/')[-1]
        if not os.path.exists(self.model_directory +'/TEST_DELONG/'+self.type_problem+'/' +self.type_image+'/'+name_model+'/'): os.mkdir(self.model_directory +'/TEST_DELONG/'+self.type_problem +'/' +self.type_image+'/'+name_model+'/')

        self.path_to_save_delong = self.model_directory +'/TEST_DELONG/'+self.type_problem+'/' +self.type_image+'/'+name_model+'/'
                
        self.model_hybrid = tf.keras.models.load_model(self.path_model_hybrid+'.hdf5')   
        self.model_images = tf.keras.models.load_model(self.path_model_images+'.hdf5')   
        
        
    def model_predict_delong(self):
        y_prediction_hybrid = self.model_hybrid.predict([self.X_test_image, self.X_test_dataf], verbose=1)
        y_prediction_images = self.model_images.predict(self.X_test_image, verbose=1)

        self.y_pred_hybrid = y_prediction_hybrid[:,0]
        self.y_pred_images = y_prediction_images[:,0]

        self.y_true = np.array(self.y_test)        

        
    def model_delong(self):
        tb = mcnemar_table(y_target=self.y_true, 
                               y_model1=self.y_pred_images, 
                               y_model2=self.y_pred_hybrid)


        chi2, p = mcnemar(ary=tb, corrected=True)
        print('chi-squared:', chi2)
        print('p-value:', p)

        brd = checkerboard_plot(tb,
                                    figsize=(3, 3),
                                    fmt='%d',
                                    col_labels=['model 2 wrong', 'model 2 right'],
                                    row_labels=['model 1 wrong', 'model 1 right'])
        plt.savefig(self.path_to_save_delong + 'CONFUSIONMATRIX_' + self.type_problem + '.png')
        plt.show()
        
        fpr, tpr, _ = roc_curve(self.y_true, self.y_pred_hybrid)
        plt.plot(fpr, tpr, linewidth=1.5)
        
        fpr, tpr, _ = roc_curve(self.y_true, self.y_pred_images)
        plt.plot(fpr, tpr, linewidth=1.5)
        plt.plot([0, 1], [0, 1], color='black', linestyle='--', label='Baseline')
        plt.legend(['H_Model', 'I_Model'], loc='best')
        plt.title('ROC Curve', size=12)
        plt.xlabel('False Positive Rate', size=10)
        plt.ylabel('True Positive Rate', size=10)
        
        plt.savefig(self.path_to_save_delong + 'CURVEROC_' + self.type_problem + '.png')
        plt.close()
        


    
    
    def DelongTest_run(self):
        if self.type_problem=='PROGNOSIS': self.load_data_PROGNOSIS()
        if self.type_problem=='ICH': self.load_data_ICH()
            
        self.load_models_delong()
        
        self.model_predict_delong()
        
        self.model_delong()

In [ ]:
#Test delong
problem = str(input("Which problem  (options: ICH_DETECTION, PROGNOSIS): "))
type_image= str(input("Which problem  (options: 3D, 2D): ")) if problem == 'DETECTION' else '3D'
HYBRIDMODEL = str(input("Hybrid Model name  (all path without '.hdf5'): "))
IMAGEMODEL=  str(input("Image Model name  (all path without '.hdf5'): "))


path_csv='/home/jovyan/IMAGE_PREPROCESSING_2D_3D/DIGITAL_CSIC_ICH/'+problem+'/CSV/' if type_image== '3D' else '/home/jovyan/IMAGE_PREPROCESSING_2D_3D/DIGITAL_CSIC_ICH/'+problem+'/'+type_image+'/CSV/'
path_images = '/home/jovyan/IMAGE_PREPROCESSING_2D_3D/DIGITAL_CSIC_ICH/'+problem+'/PREPROCESSED IMAGES/ALL_IMAGES/' if type_image== '3D' else '/home/jovyan/IMAGE_PREPROCESSING_2D_3D/DIGITAL_CSIC_ICH/'+problem+'/'+type_image+'/PREPROCESSED IMAGES/ALL_IMAGES/'


#model1='/home/jovyan/OTROS/GITHUB/RESULTS/VGG16/Hybrid/AUC0.9869_DESCONGELADO_ModeloHIBRIDOVGG16_optAdagrad_lr0.01_batch4_topGMP.hdf5'
#model2='/home/jovyan/OTROS/GITHUB/RESULTS/VGG16/Images/AUC0.9771_DESCONGELADO_ModeloVGG16_optAdagrad_lr0.01_batch4_topGMP.hdf5'

#prognosis
#hybrid=/home/jovyan/ICH_PROGNOSIS/RESULTS/Hybrid/AUC0.792MODELPROGNOSIS_TOPMODELGMP_CLASSES1_INITIALIZER_SEED2_OPTIMIZERAdagrad_DROP0.5_LR0.01_LOSSFbinary_crossentropy_BATCHSIZE6_EPOCHS100
#images=/home/jovyan/ICH_PROGNOSIS/RESULTS/Images/AUC0.72MODELPROGNOSIS_TOPMODELGMP_CLASSES1_INITIALIZER_SEED2_OPTIMIZERAdagrad_DROP0.5_LR0.01_LOSSFbinary_crossentropy_BATCHSIZE6_EPOCHS100


MyDelongTest = DelongTest(problem= problem,
                          path_csv= path_csv,
                          path_images= path_images,
                          img_width= 128,
                          img_height= 128,
                          
                          type_image= type_image,
                          model_hybrid = HYBRIDMODEL,
                          model_images = IMAGEMODEL,
                          
                          model_directory= '/home/jovyan/MODEL_ANALYSIS/'

                          )

MyDelongTest.DelongTest_run()